# Data Augmentation

### Manage dependencies

#### Install packages

In [ ]:
!python -m pip install opencv-python matplotlib tensorflow==2.10 

#### Import dependencies

In [2]:
import os
import cv2
import csv
import time
import random
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras import backend, layers, metrics

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import Xception
from tensorflow.keras.models import Model, Sequential

from tensorflow.keras.utils import plot_model
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

import seaborn as sns
import matplotlib.pyplot as plt
tf.__version__, np.__version__

('2.10.0', '1.24.2')

### Limit GPU usage 

In [3]:
# To avoid OOM error
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus: 
    tf.config.experimental.set_memory_growth(gpu, True)

### Set up environment

#### Set Home path

In [4]:
HOME = os.path.split(os.getcwd())[0]
print(HOME)
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

e:\Code\CowId


### Helper functions

In [5]:
random.seed(5)
np.random.seed(5)
tf.random.set_seed(5)
dataset_path = os.path.join(HOME, "data", "siamese_v9_augmented", 'train_val')

In [6]:
def data_aug(img, quantity):
    data = []
    for i in range(quantity):
        angle = random.randint(0, 360)
        (rows, cols) = img.shape[:2]
        M = cv2.getRotationMatrix2D((cols/2, rows/2), angle, 1)
        new_img = cv2.warpAffine(img, M, (cols, rows))
        new_img = tf.image.stateless_random_brightness(new_img, max_delta=0.15, seed=(np.random.randint(100),np.random.randint(100)))
        new_img = tf.image.stateless_random_contrast(new_img, lower=0.8, upper=1, seed=(np.random.randint(100),np.random.randint(100)))
        new_img = tf.image.stateless_random_jpeg_quality(new_img, min_jpeg_quality=90, max_jpeg_quality=100, seed=(np.random.randint(100),np.random.randint(100)))
        new_img = tf.image.stateless_random_saturation(new_img, lower=0.9,upper=1, seed=(np.random.randint(100),np.random.randint(100)))
        data.append(new_img)
    return data

Do the data augmentation to POS and ANC

In [7]:
import uuid

def create_augmentation(path, quantity):
    files = os.listdir(os.path.join(path))
    to_produce = int(quantity/len(files))
    for file_name in files:
            img_path = os.path.join(path, file_name)
            img = cv2.imread(img_path)
            augmented_images = data_aug(img, to_produce) 

            for image in augmented_images:
                cv2.imwrite(os.path.join(path, '{}.jpg'.format(uuid.uuid1())), image.numpy())

### Data Augmentation

In [12]:
# We don't need to augment negative ones, but it's good to chance the distribution a little bit

number_per_cow = 2000

for cows in os.listdir(dataset_path):
    path_to_cow = os.path.join(dataset_path, cows)
    print(path_to_cow)
    if "cow" in path_to_cow:
        create_augmentation(path_to_cow, number_per_cow)
    if "negatives" in path_to_cow:
        create_augmentation(path_to_cow, number_per_cow*len(cows))

e:\Code\CowId\data\siamese_v9_augmented\train_val\cow_1
e:\Code\CowId\data\siamese_v9_augmented\train_val\cow_10
e:\Code\CowId\data\siamese_v9_augmented\train_val\cow_11
e:\Code\CowId\data\siamese_v9_augmented\train_val\cow_12
e:\Code\CowId\data\siamese_v9_augmented\train_val\cow_13
e:\Code\CowId\data\siamese_v9_augmented\train_val\cow_14
e:\Code\CowId\data\siamese_v9_augmented\train_val\cow_15
e:\Code\CowId\data\siamese_v9_augmented\train_val\cow_16
e:\Code\CowId\data\siamese_v9_augmented\train_val\cow_17
e:\Code\CowId\data\siamese_v9_augmented\train_val\cow_18
e:\Code\CowId\data\siamese_v9_augmented\train_val\cow_19
e:\Code\CowId\data\siamese_v9_augmented\train_val\cow_2
e:\Code\CowId\data\siamese_v9_augmented\train_val\cow_20
e:\Code\CowId\data\siamese_v9_augmented\train_val\cow_21
e:\Code\CowId\data\siamese_v9_augmented\train_val\cow_23
e:\Code\CowId\data\siamese_v9_augmented\train_val\cow_25
e:\Code\CowId\data\siamese_v9_augmented\train_val\cow_26
e:\Code\CowId\data\siamese_v9_aug

AttributeError: 'NoneType' object has no attribute 'shape'